# Practice №11
## by Anton Kondrashov
##### HSE, CS, BSE 141(1)

### Dependencies

In [1]:
%precision 6
%load_ext line_profiler
%load_ext autoreload
%autoreload 1
import sklearn
import pandas as pd
import numpy as np
import codecs
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Datasets

In [2]:
def unpack_dataset(path, to=None):
    dicts = []
    
    if to is not None:
        dicts = list(to.to_dict().values())
#     with codecs.open(path, 'r', encoding=encoding) as f:
#         for line in f:
#             dicts.append(eval(line))
    with open(path) as f:
        for line in f:
            single_dict = eval(line)
            dicts.append(single_dict)
    
    return pd.DataFrame(dicts)

def labelize(column_name, dataset):
    le = preprocessing.LabelEncoder()
    le.fit(dataset[column_name])
    dataset[column_name] = le.transform(dataset[column_name])
    
def process_runtime(time):
    if re.findall("\d*", time)[0] == '':
        return 0 
    else:
        return int(re.findall("\d*", time)[0])

# def replace_dates(dataset):
#     extension_set = pd.DataFrame(columns=['year', 'month', 'day'])
#     date['time'].apply(lambda x: x[:3])

In [6]:
main_path = '/Users/kondranton/Desktop/Datasets'
movie_descriptions = unpack_dataset(main_path + '/movie_descriptions.txt')

feedbacks = []
with open(main_path + '/train_feedback.txt') as f:
        for line in f:
            feedback = eval(line)
            feedbacks.append(feedback)
            
with open(main_path + '/test_feedback.txt') as f:
        for line in f:
            feedback = eval(line)
            feedbacks.append(feedback)
            
feedbacks = pd.DataFrame(feedbacks)

In [7]:
len(feedbacks)

133906

In [18]:
movie_descriptions

,MovieId,actors,awards,country,director,genre,language,plot,rated,released,runtime,synopsis,type,writer,year
0,349,"Jimmy Smits, Marg Helgenberger, John Ashton, A...",N/A,"USA, New Zealand",N/A,"Horror, Sci-Fi",English,A buried UFO slowly turns local inhabitants in...,R,09 May 1993,181 min,\n\nPart OneBobbi Anderson (Marg Helgenberger)...,series,N/A,1993
1,2537,"Spencer Tracy, Sidney Poitier, Katharine Hepbu...",Won 2 Oscars. Another 7 wins & 22 nominations.,USA,Stanley Kramer,"Comedy, Drama",English,"After a period of vacation in Hawaii, Joanna &...",UNRATED,12 Dec 1967,108 min,"\n\nThe movie concerns Joanna Drayton, a young...",movie,William Rose,1967
2,2186,"Tom Hanks, Leslie Zemeckis, Eddie Deezen, Nona...",Nominated for 3 Oscars. Another 5 wins & 22 no...,USA,Robert Zemeckis,"Animation, Adventure, Family",English,This is the story of a young hero boy on Chris...,G,10 Nov 2004,100 min,\n\nTom Hanks plays the train conductor of a m...,movie,"Chris Van Allsburg (book), Robert Zemeckis (sc...",2004
3,2534,"Nicole Kidman, Matthew Broderick, Bette Midler...",3 wins.,USA,Frank Oz,"Comedy, Sci-Fi, Thriller",English,"Joanna Eberhart, a wildly successful president...",PG-13,11 Jun 2004,93 min,\n\nJoanna Eberhart (Nicole Kidman) is a top N...,movie,"Ira Levin (book), Paul Rudnick (screenplay)",2004
4,827,"Colin Farrell, Alicja Bachleda, Dervla Kirwan,...",6 wins & 6 nominations.,"Ireland, USA",Neil Jordan,"Drama, Mystery, Romance","English, Romanian, French","On the coast of Cork, Syracuse is a divorced f...",PG-13,05 Mar 2010,111 min,\n\nONDINE is a lyrical modern fairy tale that...,movie,Neil Jordan,2009
5,2975,"Mel Gibson, Joanne Samuel, Hugh Keays-Byrne, S...",4 wins & 6 nominations.,Australia,George Miller,"Action, Adventure, Sci-Fi",English,Taking place in a dystopian Australia in the n...,R,21 Mar 1980,88 min,\n\nThe film is set in the near future of a bl...,movie,"James McCausland (screenplay), George Miller (...",1979
6,2569,"Michael Keaton, Gedde Watanabe, George Wendt, ...",1 win.,USA,Ron Howard,"Comedy, Drama",English,When a Japanese car company buys an American p...,PG-13,14 Mar 1986,111 min,"\n\nIn a town in rural Pennsylvania, the car f...",movie,"Edwin Blum (story), Lowell Ganz (story), Babal...",1986
7,1441,"William Holden, Ernest Borgnine, Robert Ryan, ...",Nominated for 2 Oscars. Another 5 wins & 4 nom...,USA,Sam Peckinpah,"Action, Adventure, Western","English, Spanish, German",In the Wild Bunch the movie opens with a group...,R,07 Aug 1969,135 min,"\n\nThe movie takes place in 1913, during the ...",movie,"Walon Green (screenplay), Sam Peckinpah (scree...",1969
8,2002,"Tygh Runyan, Dominique Swain, Shannyn Sossamon...",1 nomination.,USA,Monte Hellman,"Romance, Thriller",English,There's a murky tenuous balance between realit...,R,07 Apr 2011,121 min,\n\nTheres a murky tenuous balance between rea...,movie,Steven Gaydos,2010
9,1389,"Jack Black, Jason Segel, Emily Blunt, Amanda Peet",3 nominations.,USA,Rob Letterman,"Adventure, Comedy, Family",English,Travel writer Lemuel Gulliver takes an assignm...,PG,25 Dec 2010,85 min,\n\nLemuel Gulliver (Jack Black) is happy to c...,movie,"Joe Stillman (screenplay), Nicholas Stoller (s...",2010


### Preprocessing

In [22]:
# digitalize profileNames
le = preprocessing.LabelEncoder()
le.fit(movie_descriptions['rated'])
movie_descriptions['rated'] = le.transform(movie_descriptions['rated'])

movie_descriptions['runtime'] = movie_descriptions['runtime'].apply(process_runtime)
labelize('rated', movie_descriptions)
labelize('type', movie_descriptions)

labelize('time', train_feedback)
train_feedback.drop(['id'], axis=1, inplace=True)
train_feedback

,MovieId,UserId,profileName,score,summary,text,time
0,4043,7022,"Jon Stevens ""Journalist""",4,Great action film,This is one of my all time favorite American a...,4117
1,2347,2996,Michelle Sutton,5,Movie is way better than the book!,"First, I have to say I'm incredibly biased tow...",4243
2,89,795,Jay Chen,4,A very good film that could have been great,The Abyss is one of those movies you can watch...,4411
3,407,13617,Jack D. Lowry,4,Comedy for all ages,Happy Gilmore is a feel good comedy for all ag...,4758
4,627,9254,Philip Dunaway,4,better pic quality than standard dvd version,I enjoyed this movie even more on Blu-ray than...,3762
5,1280,10720,Daisy Ghostly,3,Train of terror,"Think what could've been, if only Hammer had m...",416
6,304,5945,"""vintage_seafoam""",5,even my thesaurus can't help me describe this,If you've never seen this movie and are planni...,1052
7,1698,4797,Chanta Rose,5,Amazing,I am not a fan of drugs. Drug use offends me g...,2406
8,2662,5716,"J. Kramer ""Carpenter L..U 45""",4,very good job by rob zombie,"Very cool movie, he actually brought back 70's...",2097
9,1517,2232,MadMacs,4,Challenging Understated Performance,I'm going to start by saying from the get-go t...,4504


In [8]:
Z=pd.merge(feedbacks,movie_descriptions,on='MovieId')

In [9]:
np.random.seed(0)
Z = Z.sample(frac=1.0).reset_index(drop=True)

In [10]:
Z.rename(columns={'score':'y'}, inplace=True)

In [11]:
Z

,MovieId,UserId,id,profileName,y,summary,text,time,actors,awards,country,director,genre,language,plot,rated,released,runtime,synopsis,type,writer,year
0,2210,5552,93554,"Craig Chalquist, PhD, author of TERRAPSYCHOLO...",5.0,Where is healing to be found?,"In well-paid experts, or in those marginalized, imprisoned people we pass off as beyond redempti...",2001-08-12 03:00:00,"William Powell, Myrna Loy, Maureen O'Sullivan, Nat Pendleton",Nominated for 4 Oscars. Another 2 wins.,USA,W.S. Van Dyke,"Comedy, Crime, Mystery",English,"After a four year absence, one time detective Nick Charles returns to New York with his new wife...",NOT RATED,25 May 1934,91 min,"\n\nFamous New York detective Nick Charles met and fell in love with Nora, an heiress from an ol...",movie,"Albert Hackett (screen play), Frances Goodrich (screen play), Dashiell Hammett (from the novel by)",1934
1,3446,1012,19548,S. Edmondson,5.0,GOOD SCI FI,A DUPLICATION MISTAKE OF MINE BUT THIS ONE'S NOT A DOUBLE FEATURE DVD LIKE MY OTHER ONE. GOOD S...,2010-03-19 03:00:00,"Carlos Gardel, Rosita Moreno, Tito Lusiardo, Manuel Peluffo",N/A,"USA, Argentina",John Reinhardt,"Drama, Musical",Spanish,Carlos Argüelles is the son of a wealthy man whose only interests in life are business and makin...,N/A,24 Oct 1935,82 min,\n\nJulio Arguelles is the son of a wealthy Buenos Aires businessman. He wants to marry Margarit...,movie,Alfredo Le Pera,1935
2,1211,4911,11899,Don Talon,5.0,Perfect - This is a must see.,I dont even know what to say. This movie has made me seriously interested in this classical musi...,2006-04-05 03:00:00,"Anne Baxter, Richard Conte, Ann Sothern, Raymond Burr",N/A,USA,Fritz Lang,"Crime, Drama, Film-Noir",English,A telephone operator ends up drunk and at the mercy of a cad in his apartment. The next morning ...,NOT RATED,28 Mar 1953,90 min,\n\nCasey Mayo (Richard Conte) is the star reporter of the newspaper world and everybody knows h...,movie,"Charles Hoffman (screenplay), Vera Caspary (story)",1953
3,133,5027,20186,S William Shaw,NaN,Very Enjoyable,"I have to admit, being a veteran of waaaaay too many children's flicks, especially those with an...",2008-08-31 03:00:00,"Gene Evans, André Morell, John Turner, Leigh Madison",N/A,"UK, USA","Douglas Hickox, Eugène Lourié","Drama, Fantasy, Horror",English,Marine atomic tests cause changes in the ocean's ecosystem resulting in dangerous blobs of radia...,NOT RATED,03 Mar 1959,80 min,\n\nWe see a series of nuclear blasts followed by footage of men in radiation suits measuring th...,movie,"Robert Abel (story), Allen Adler (story), Eugène Lourié (screenplay), Daniel James (screenplay)",1959
4,3211,8029,130538,"TANTRUM!!!! ""...no longer Maria!""",NaN,"Sergio Leone's masterful version of Akira Kurosawa's ""Yojimbo"", was the first important Spaghett...","This epic cinema classic in western, filmed in Italy, West Germany and Spain, was the first majo...",2008-03-14 03:00:00,"Liam Neeson, Pierce Brosnan, Michael Wincott, Xander Berkeley",1 win & 1 nomination.,USA,David Von Ancken,"Action, Drama, Thriller",English,"In the 1860s, five men have been tracking a sixth across Nevada for more than two weeks. They sh...",R,13 Apr 2007,115 min,\n\nThe film is set after the end of the American Civil War. Gideon (Pierce Brosnan) is hiding i...,movie,"David Von Ancken, Abby Everett Jaques",2006
5,2181,15209,37243,"J. Irps ""krazejjklown""",NaN,great job robert de niro,i thought robert de niro did an excellent job as frankensteins monster in this tale. they did an...,2005-03-07 03:00:00,"William Hurt, Kathleen Turner, Richard Crenna, Ted Danson",Nominated for 1 Golden Globe. Another 5 nominations.,USA,Lawrence Kasdan,"Crime, Drama, Romance",English,Ned Racine is a seedy small town lawyer in Florida. During a searing heatwave he's picked up by ...,R,28 Aug 1981,113 min,\n\nMary Ann Simpson (Kathleen Turner) and Matty Tyler (Kim Zimmer) graduate together from Wheat...,movie,Lawrence Kasdan,1981
6,1127,15704,130601,"Movie Buff ""Lynn""",2.0

In [40]:
train_sels = ~np.isnan(Z.y)
test_sels = np.isnan(Z.y)

train_inds = np.where(train_sels)
test_inds = np.where(test_sels)

In [23]:
corpus = Z.text

vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)

Y=Z.y.values

In [24]:
clf = sklearn.svm.LinearSVC(C=1)
clf.fit(X[train_inds], Y[train_inds])

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [25]:
Y_hat = clf.predict(X[test_inds])

In [72]:
content = pd.concat([pd.DataFrame(Z.id.values[test_inds]), pd.DataFrame(Y_hat)], axis=1)
content.columns = ['ReviewId', 'Score']

In [78]:
solution = pd.DataFrame(content, dtype=int)
solution.to_csv(main_path + '/solution.csv',index=False)

In [65]:
vectorizer = TfidfVectorizer(min_df=0.01, stop_words='english')
text_matrix = pd.DataFrame(vectorizer.fit_transform(train_feedback['text'] + train_feedback['summary'] + train_feedback['profileName']).toarray())

In [66]:
#some code is suitable for future use
train_feedback = pd.concat([train_feedback[['score', 'MovieId', 'UserId', 'time']], text_matrix], axis=1)

train_percentage = 50000/len(train_feedback)
mask = np.random.rand(len(train_feedback)) < train_percentage
train_set = train_feedback[mask]
test_set = train_feedback[~mask]

In [67]:
train_x = train_set[[x for x in list(train_set.columns) if x != 'score']]
train_y = train_set[['score']].as_matrix()
test_x = test_set[[x for x in list(train_set.columns) if x != 'score']]
test_y = test_set[['score']].as_matrix()

In [68]:
regressor = RandomForestRegressor(n_jobs=-1, **{'max_features': 0.07, 'min_samples_leaf': 2})
regressor.fit(train_x, [y[0] for y in train_y])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.07, max_leaf_nodes=None, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [69]:
regressor.score(test_x, [x[0] for x in test_y])

0.42911582910348933

In [70]:
regressor.feature_importances_[:9]

array([ 0.00896058,  0.00910815,  0.01034011,  0.00041632,  0.00015229,
        0.00019456,  0.00019262,  0.00039515,  0.0004401 ])

In [71]:
importances = pd.concat([pd.DataFrame(train_x.columns), pd.DataFrame(regressor.feature_importances_)], axis=1)

In [72]:
importances.columns = ['feat', 'import']
important_indexes = importances.sort_values('import', ascending=False)[:9]['feat'].as_matrix()

In [73]:
names = vectorizer.get_feature_names()
[names[i] for i in important_indexes]

TypeError: list indices must be integers or slices, not str

### Conclusion

As we can see, these words have the most significant impact on a review score. The words express someone's opinion, so they can be intepreted as a movie empirical score. 

In [74]:
names

['10',
 '100',
 '12',
 '13',
 '15',
 '20',
 '30',
 '40',
 '50',
 '70',
 '80',
 '90',
 'ability',
 'able',
 'absolute',
 'absolutely',
 'academy',
 'accurate',
 'act',
 'acted',
 'acting',
 'action',
 'actions',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adaptation',
 'add',
 'added',
 'addition',
 'adds',
 'admit',
 'adult',
 'adults',
 'adventure',
 'age',
 'agent',
 'ages',
 'ago',
 'agree',
 'ahead',
 'air',
 'al',
 'alan',
 'alien',
 'aliens',
 'alive',
 'allowed',
 'alot',
 'amazing',
 'amazon',
 'america',
 'american',
 'americans',
 'amp',
 'amusing',
 'animated',
 'animation',
 'annoying',
 'answer',
 'anthony',
 'anti',
 'anymore',
 'apart',
 'apparently',
 'appeal',
 'appear',
 'appearance',
 'appears',
 'appreciate',
 'aren',
 'army',
 'art',
 'aside',
 'ask',
 'asked',
 'aspect',
 'aspects',
 'atmosphere',
 'attack',
 'attempt',
 'attempts',
 'attention',
 'audience',
 'audiences',
 'audio',
 'author',
 'available',
 'average',
 'avoid',
 'award',
 